# Creating the Initial Slice


In [9]:
output_table = fablib.list_sites()

Name,State,Address,Location,PTP Capable,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,Ram Available,Ram Capacity,Ram Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,P4-Switch Available,P4-Switch Capacity,P4-Switch Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated,A30 Available,A30 Capacity,A30 Allocated,A40 Available,A40 Capacity,A40 Allocated,U280 Available,U280 Capacity,U280 Allocated
AMST,Active,"Science Park 904, 1098 XH Amsterdam","(52.3544941, 4.9557553)",True,3,6,132,384,252,634,1434,800,58818,64068,5250,313,381,68,0,0,0,2,3,1,2,2,0,10,10,0,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0
STAR,Active,"710 North Lakeshore Drive, 60611","(42.2359989, -88.1575427)",True,6,12,364,768,404,1532,2820,1288,101976,109696,7720,663,762,99,1,1,0,1,2,1,6,6,0,18,20,2,2,6,4,0,6,6,0,0,0,0,0,0,1,1,0
RUTG,Active,"120 Avenue E,Piscataway, New Jersey","(40.5224962, -74.4405719)",True,5,10,436,640,204,1582,2390,808,102143,107463,5320,540,635,95,0,0,0,2,2,0,4,4,0,16,16,0,0,0,0,0,0,0,8,8,0,0,0,0,1,1,0
TACC,Active,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261807)",False,5,10,124,640,516,646,2390,1744,97743,107463,9720,573,635,62,0,0,0,2,2,0,4,4,0,16,16,0,0,4,4,3,6,3,0,0,0,0,0,0,1,1,0
NCSA,Active,"1725 S Oak St.,Champaign, IL 61820","(40.09584, -88.2415369)",True,3,6,302,384,82,1106,1434,328,60151,64161,4010,340,381,41,0,0,0,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0,0,0,0,0,0,0,1,1,0
DALL,Active,"1950 N Stemmons Fwy,Dallas TX 75207","(32.7990816, -96.8206903)",True,3,6,156,384,228,714,1434,720,58921,64161,5240,345,381,36,0,0,0,1,2,1,2,2,0,10,10,0,2,2,0,3,3,0,0,0,0,0,0,0,1,1,0
MAX,Active,"4161 Fieldhouse Drive,College Park,MD 20742","(38.9886345, -76.9434794)",True,5,10,502,640,138,1934,2390,456,102753,107463,4710,591,635,44,0,0,0,2,2,0,4,4,0,16,16,0,4,4,0,3,6,3,0,0,0,0,0,0,1,1,0
SALT,Active,"572 Delong Street,Salt Lake City, UT 84104","(40.7570751, -111.9534664)",False,3,6,252,384,132,906,1434,528,58861,64161,5300,337,381,44,0,0,0,2,2,0,2,2,0,10,10,0,2,2,0,2,3,1,0,0,0,0,0,0,1,1,0
UCSD,Active,"10100 Hopkins Drive,CA 92093","(32.8886802, -117.239324)",True,5,10,248,640,392,1126,2390,1264,44887,58717,13830,563,635,72,1,1,0,2,2,0,4,4,0,14,16,2,4,4,0,1,6,5,0,0,0,0,0,0,1,1,0
MICH,Active,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",True,3,6,244,384,140,978,1434,456,59641,64161,4520,333,381,48,1,1,0,1,2,1,2,2,0,10,10,0,2,2,0,2,3,1,0,0,0,0,0,0,1,1,0


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

In [2]:
slice = fablib.new_slice(name="INTProject")

In [3]:
#End host 1
site1='SRI'   #SRI
#Switches
site2='UCSD'  #UCSD
site3='STAR'  #STAR
site4='MAX'  #ATLA
#End host 2
site5='RUTG'  #WASH

print (f'The selected sites are {site1}, {site2}, {site3}, {site4}, {site5}') 

The selected sites are SRI, UCSD, STAR, MAX, RUTG


## Create Nodes


In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site5, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
monitor1 = slice.add_node(name="monitor1", 
                      site=site2, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
monitor2 = slice.add_node(name="monitor2", 
                      site=site4, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]
monitor1_iface = monitor1.add_component(model='NIC_Basic').get_interfaces()[0]
monitor2_iface = monitor2.add_component(model='NIC_Basic').get_interfaces()[0]

## P4 Switches Nodes

In [5]:
switch1 = slice.add_node(name="switch1", 
                      site=site2, 
                      cores=16, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')
switch2 = slice.add_node(name="switch2", 
                      site=site3, 
                      cores=16, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')
switch3 = slice.add_node(name="switch3", 
                      site=site4, 
                      cores=16, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')
switch_iface1 = switch1.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch1.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch_iface3 = switch2.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface4 = switch2.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]
switch_iface5 = switch3.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface6 = switch3.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

switch_sink_iface1 = switch1.add_component(model='NIC_Basic', name='net3_nic').get_interfaces()[0]
switch_sink_iface2 = switch3.add_component(model='NIC_Basic', name='net3_nic').get_interfaces()[0]

## Connecting Sites

In [6]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch_iface1])
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, switch_iface3])
net3 = slice.add_l2network(name='net3', interfaces=[switch_iface4, switch_iface5])
net4 = slice.add_l2network(name='net4', interfaces=[switch_iface6, server2_iface])

net5 = slice.add_l2network(name='net5', interfaces=[switch_sink_iface1, monitor1_iface])
net6 = slice.add_l2network(name='net6', interfaces=[switch_sink_iface2, monitor2_iface])


## Submit Slice

In [7]:
slice.submit();


Retry: 11, Time: 314 sec


ID,aa47c4b8-78d6-4bbd-ac8a-2e9d6e0da5d5
Name,INTProject
Lease Expiration (UTC),2024-12-12 03:19:24 +0000
Lease Start (UTC),2024-12-11 03:19:24 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
d0e5fbcd-4936-498e-9328-16a30d1fae0e,monitor1,4,8,100,default_ubuntu_20,qcow2,ucsd-w4.fabric-testbed.net,UCSD,ubuntu,132.249.252.184,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.184,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
431d0610-2809-402a-b12e-d01936d3a3e9,monitor2,4,8,100,default_ubuntu_20,qcow2,max-w3.fabric-testbed.net,MAX,ubuntu,2001:468:c00:ffc4:f816:3eff:fea0:dcd9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fea0:dcd9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
93249cbb-b503-4606-b1fa-f2ca7181b6d8,server1,4,8,100,default_ubuntu_20,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.82,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.82,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
38426ce8-2246-41a9-9d62-1e9c24a0811a,server2,4,8,100,default_ubuntu_20,qcow2,rutg-w3.fabric-testbed.net,RUTG,ubuntu,2620:0:d61:4101:f816:3eff:feeb:a487,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:d61:4101:f816:3eff:feeb:a487,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5af57595-62eb-43dd-97a3-ce14ae0ef08d,switch1,16,16,100,default_ubuntu_20,qcow2,ucsd-w4.fabric-testbed.net,UCSD,ubuntu,132.249.252.164,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.164,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f0a3c238-3953-4883-9a89-e30d138e7343,switch2,16,16,100,default_ubuntu_20,qcow2,star-w1.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fea7:8621,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fea7:8621,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
00571379-40f9-4e2b-83c6-c4adae2dd47d,switch3,16,16,100,default_ubuntu_20,qcow2,max-w3.fabric-testbed.net,MAX,ubuntu,2001:468:c00:ffc4:f816:3eff:fedb:5f2f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fedb:5f2f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
d65df12b-bc6b-4fde-bace-18ff3ebfca56,net1,L2,L2STS,None,None,None,Active,
86b6943c-b364-4694-8cae-1810f5e88ade,net2,L2,L2STS,None,None,None,Active,
2ea32f4d-9d25-477e-a7f9-40db85f6f1db,net3,L2,L2STS,None,None,None,Active,
37d681d6-8fb9-4755-a638-72b8ba79e646,net4,L2,L2STS,None,None,None,Active,
91931fac-7bf6-4071-a544-e1110e1e1f08,net5,L2,L2Bridge,UCSD,None,None,Active,
a4a71f3d-c1c3-48a1-9648-c4668fc3a3ce,net6,L2,L2Bridge,MAX,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
server1-None-p1,p1,server1,net1,100,config,,02:3D:E1:49:C3:3D,enp7s0,enp7s0,fe80::3d:e1ff:fe49:c33d,4,HundredGigE0/0/0/9
server2-None-p1,p1,server2,net4,100,config,,0A:90:50:1E:35:87,enp7s0,enp7s0,fe80::890:50ff:fe1e:3587,4,HundredGigE0/0/0/9
monitor1-None-p1,p1,monitor1,net5,100,config,,0A:58:8B:A8:33:27,enp7s0,enp7s0,fe80::858:8bff:fea8:3327,4,HundredGigE0/0/0/11
monitor2-None-p1,p1,monitor2,net6,100,config,,02:57:E9:7C:EC:0D,enp7s0,enp7s0,fe80::57:e9ff:fe7c:ec0d,4,HundredGigE0/0/0/9
switch1-net1_nic-p1,p1,switch1,net1,100,config,,0A:CA:AF:DF:84:03,enp7s0,enp7s0,fe80::8ca:afff:fedf:8403,4,HundredGigE0/0/0/11
switch1-net3_nic-p1,p1,switch1,net5,100,config,,16:E7:68:5C:0E:B8,enp9s0,enp9s0,fe80::14e7:68ff:fe5c:eb8,4,HundredGigE0/0/0/11
switch1-net2_nic-p1,p1,switch1,net2,100,config,,16:71:10:B0:88:76,enp8s0,enp8s0,fe80::1471:10ff:feb0:8876,4,HundredGigE0/0/0/11
switch2-net2_nic-p1,p1,switch2,net3,100,config,,02:5C:96:36:FB:07,enp7s0,enp7s0,fe80::5c:96ff:fe36:fb07,6,HundredGigE0/0/0/5
switch2-net1_nic-p1,p1,switch2,net2,100,config,,06:D3:3E:48:FE:3B,enp8s0,enp8s0,fe80::4d3:3eff:fe48:fe3b,6,HundredGigE0/0/0/5
switch3-net1_nic-p1,p1,switch3,net3,100,config,,3E:E7:7D:81:47:03,enp8s0,enp8s0,fe80::3ce7:7dff:fe81:4703,4,HundredGigE0/0/0/9



Time to print interfaces 359 seconds


## Configure Sites

In [8]:
#Install bmv2 and net tools on switches
for x in range (1,4):
    print(x)
    switch = slice.get_node(name="switch" + str(x))     
    switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
    stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)
    stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

1
2
3


In [9]:
#Net tools and scapy on servers
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")

monitor1 = slice.get_node(name="monitor1")
monitor2 = slice.get_node(name="monitor2")

stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)

stdout, stderr = monitor1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = monitor2.execute(f'sudo apt-get install -y net-tools', quiet=True)

stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

stdout, stderr = monitor1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = monitor2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)


In [10]:
#Interfaces
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net4') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')


monitor1_iface = monitor1.get_interface(network_name='net5') 
monitor1_iface_name = monitor1_iface.get_device_name()
print(f'monitor1_iface: {monitor1_iface_name}')

monitor2_iface = monitor2.get_interface(network_name='net6') 
monitor2_iface_name = monitor2_iface.get_device_name()
print(f'monitor2_iface: {monitor2_iface_name}')

switch1 = slice.get_node(name="switch1")
switch2 = slice.get_node(name="switch2")
switch3 = slice.get_node(name="switch3")


switch1_iface1 = switch1.get_interface(network_name='net1') 
switch1_iface1_name = switch1_iface1.get_device_name()
print(f'switch_iface1: {switch1_iface1_name}')
stdout, stderr = switch.execute(f'sudo ip link set {switch1_iface1_name} mtu 2000')

switch1_iface2 = switch1.get_interface(network_name='net2') 
switch1_iface2_name = switch1_iface2.get_device_name()
print(f'switch_iface2: {switch1_iface2_name}')
stdout, stderr = switch1.execute(f'sudo ip link set {switch1_iface2_name} mtu 2000')


switch_sink_iface1 = switch1.get_interface(network_name='net5') 
switch_sink_iface1_name = switch_sink_iface1.get_device_name()
print(f'switch_sink_iface1: {switch_sink_iface1_name}')
stdout, stderr = switch1.execute(f'sudo ip link set {switch_sink_iface1_name} mtu 2000')

switch2_iface1 = switch2.get_interface(network_name='net2') 
switch2_iface1_name = switch2_iface1.get_device_name()
print(f'switch_iface3: {switch2_iface1_name}')
stdout, stderr = switch2.execute(f'sudo ip link set {switch2_iface1_name} mtu 2000')

switch2_iface2 = switch2.get_interface(network_name='net3') 
switch2_iface2_name = switch2_iface2.get_device_name()
print(f'switch_iface4: {switch2_iface2_name}')
stdout, stderr = switch2.execute(f'sudo ip link set {switch2_iface2_name} mtu 2000')


switch3_iface1 = switch3.get_interface(network_name='net3') 
switch3_iface1_name = switch3_iface1.get_device_name()
print(f'switch_iface5: {switch3_iface1_name}')
stdout, stderr = switch3.execute(f'sudo ip link set {switch3_iface1_name} mtu 2000')

switch3_iface2 = switch3.get_interface(network_name='net4') 
switch3_iface2_name = switch3_iface2.get_device_name()
print(f'switch_iface6: {switch3_iface2_name}')
stdout, stderr = switch3.execute(f'sudo ip link set {switch3_iface2_name} mtu 2000')

switch_sink_iface2 = switch3.get_interface(network_name='net6') 
switch_sink_iface2_name = switch_sink_iface2.get_device_name()
print(f'switch_sink_iface2: {switch_sink_iface2_name}')
stdout, stderr = switch3.execute(f'sudo ip link set {switch_sink_iface2_name} mtu 2000')


server1_iface: enp7s0
server2_iface: enp7s0
monitor1_iface: enp7s0
monitor2_iface: enp7s0
switch_iface1: enp7s0
switch_iface2: enp8s0
switch_sink_iface1: enp9s0
switch_iface3: enp8s0
switch_iface4: enp7s0
switch_iface5: enp8s0
switch_iface6: enp9s0
switch_sink_iface2: enp7s0


In [11]:
#Turn interfaces up
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)

stdout, stderr = monitor1.execute(f'sudo ip link set dev {monitor1_iface_name} up', quiet=True)
stdout, stderr = monitor2.execute(f'sudo ip link set dev {monitor2_iface_name} up', quiet=True)

stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface1_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch1_iface2_name} up', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip link set dev {switch_sink_iface1_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface1_name} up', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip link set dev {switch2_iface2_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface1_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch3_iface2_name} up', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip link set dev {switch_sink_iface2_name} up', quiet=True)

In [12]:
#Hardcode MAC addresses
server1_iface_MAC = '00:00:00:00:00:01'
switch1_iface1_MAC = '00:00:00:00:00:02'
switch1_iface2_MAC = '00:00:00:00:00:03'
switch2_iface1_MAC = '00:00:00:00:00:04'
switch2_iface2_MAC = '00:00:00:00:00:05'
switch3_iface1_MAC = '00:00:00:00:00:06'
switch3_iface2_MAC = '00:00:00:00:00:07'
server2_iface_MAC = '00:00:00:00:00:08'

switch_sink_iface1_MAC = '00:00:00:00:00:09'
monitor1_iface_MAC = '00:00:00:00:00:0A'

switch_sink_iface2_MAC = '00:00:00:00:00:0B'
monitor2_iface_MAC = '00:00:00:00:00:0C'


In [13]:
#IP addresses
server1_switch1_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.1/24'
switch1_ip1 = '192.168.1.2/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} {switch1_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface1_name} hw ether {switch1_iface1_MAC}')

switch1_switch2_subnet = "192.168.2.0/24"
switch1_ip2 = '192.168.2.1/24'
switch2_ip1 = '192.168.2.2/24'

stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface2_name} {switch1_ip2}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface1_name} {switch2_ip1}')

stdout, stderr = switch1.execute(f'sudo ifconfig {switch1_iface2_name} hw ether {switch1_iface2_MAC}')
stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface1_name} hw ether {switch2_iface1_MAC}')

switch2_switch3_subnet = "192.168.3.0/24"
switch2_ip2 = '192.168.3.1/24'
switch3_ip1 = '192.168.3.2/24'

stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} {switch2_ip2}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface1_name} {switch3_ip1}')

stdout, stderr = switch2.execute(f'sudo ifconfig {switch2_iface2_name} hw ether {switch2_iface2_MAC}')
stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface1_name} hw ether {switch3_iface1_MAC}')

switch3_server2_subnet = "192.168.4.0/24"
switch3_ip2 = '192.168.4.1/24'
server2_ip = '192.168.4.2/24'

stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface2_name} {switch3_ip2}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')

stdout, stderr = switch3.execute(f'sudo ifconfig {switch3_iface2_name} hw ether {switch3_iface2_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')

switch1_monitor1_subnet = "192.168.5.0/24"
switch1_ip3 = '192.168.5.1/24'
monitor1_ip = '192.168.5.2/24'

stdout, stderr = switch1.execute(f'sudo ifconfig {switch_sink_iface1_name} {switch1_ip3}')
stdout, stderr = monitor1.execute(f'sudo ifconfig {monitor1_iface_name} {monitor1_ip}')

stdout, stderr = switch1.execute(f'sudo ifconfig {switch_sink_iface1_name} hw ether {switch_sink_iface1_MAC}')
stdout, stderr = monitor1.execute(f'sudo ifconfig {monitor1_iface_name} hw ether {monitor1_iface_MAC}')

switch3_monitor2_subnet = "192.168.6.0/24"
switch3_ip3 = '192.168.6.1/24'
monitor2_ip = '192.168.6.2/24'

stdout, stderr = switch3.execute(f'sudo ifconfig {switch_sink_iface2_name} {switch3_ip3}')
stdout, stderr = monitor2.execute(f'sudo ifconfig {monitor2_iface_name} {monitor2_ip}')

stdout, stderr = switch3.execute(f'sudo ifconfig {switch_sink_iface2_name} hw ether {switch_sink_iface2_MAC}')
stdout, stderr = monitor2.execute(f'sudo ifconfig {monitor2_iface_name} hw ether {monitor2_iface_MAC}')



In [14]:
#Enable forwards
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch1.execute(command, quiet=True)
stdout, stderr = switch2.execute(command, quiet=True)
stdout, stderr = switch3.execute(command, quiet=True)

In [15]:
#Delete Routing

stdout, stderr = switch1.execute(f'sudo ip route del {server1_switch1_subnet}', quiet=True)
stdout, stderr = switch1.execute(f'sudo ip route del {switch3_server2_subnet}', quiet=True)

stdout, stderr = switch2.execute(f'sudo ip route del {server1_switch1_subnet}', quiet=True)
stdout, stderr = switch2.execute(f'sudo ip route del {switch3_server2_subnet}', quiet=True)

stdout, stderr = switch3.execute(f'sudo ip route del {server1_switch1_subnet}', quiet=True)
stdout, stderr = switch3.execute(f'sudo ip route del {switch3_server2_subnet}', quiet=True)

In [16]:
#Configure Routing
gw1 = switch1_ip1.split('/')[0]
gw2 = switch3_ip2.split('/')[0]
print(gw1, gw2)
stdout, stderr = server1.execute(f'sudo ip route add {switch3_server2_subnet} via {gw1}')
stdout, stderr = server2.execute(f'sudo ip route add {server1_switch1_subnet} via {gw2}')
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch1_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch3_iface2_MAC}')
gw1 = switch1_ip3.split('/')[0]
gw2 = switch3_ip3.split('/')[0]
stdout, stderr = monitor1.execute(f'sudo arp -s {gw1} {switch_sink_iface1_MAC}')
stdout, stderr = monitor2.execute(f'sudo arp -s {gw2} {switch_sink_iface2_MAC}')

"""
gw1 = server1_ip.split('/')[0]
gw2 = switch2_ip1.split('/')[0]
stdout, stderr = switch1.execute(f'sudo arp -s {gw1} {server1_iface_MAC}')
stdout, stderr = switch1.execute(f'sudo arp -s {gw2} {switch2_iface1_MAC}')

gw1 = switch2_ip1.split('/')[0]
gw2 = switch2_ip2.split('/')[0]
stdout, stderr = switch1.execute(f'sudo ip route add {switch3_server2_subnet} via {gw1}')
stdout, stderr = switch3.execute(f'sudo ip route add {server1_switch1_subnet} via {gw2}')

gw1 = switch3_ip1.split('/')[0]
gw2 = switch1_ip2.split('/')[0]
stdout, stderr = switch2.execute(f'sudo ip route add {switch3_server2_subnet} via {gw1}')
stdout, stderr = switch2.execute(f'sudo ip route add {server1_switch1_subnet} via {gw2}')

"""

192.168.1.2 192.168.4.1


"\ngw1 = server1_ip.split('/')[0]\ngw2 = switch2_ip1.split('/')[0]\nstdout, stderr = switch1.execute(f'sudo arp -s {gw1} {server1_iface_MAC}')\nstdout, stderr = switch1.execute(f'sudo arp -s {gw2} {switch2_iface1_MAC}')\n\ngw1 = switch2_ip1.split('/')[0]\ngw2 = switch2_ip2.split('/')[0]\nstdout, stderr = switch1.execute(f'sudo ip route add {switch3_server2_subnet} via {gw1}')\nstdout, stderr = switch3.execute(f'sudo ip route add {server1_switch1_subnet} via {gw2}')\n\ngw1 = switch3_ip1.split('/')[0]\ngw2 = switch1_ip2.split('/')[0]\nstdout, stderr = switch2.execute(f'sudo ip route add {switch3_server2_subnet} via {gw1}')\nstdout, stderr = switch2.execute(f'sudo ip route add {server1_switch1_subnet} via {gw2}')\n\n"

In [17]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="INTProject")
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
switch1 = slice.get_node(name='switch1')        
switch2 = slice.get_node(name='switch2')        
switch3 = slice.get_node(name='switch3')        
monitor1 = slice.get_node(name='monitor1')        
monitor2 = slice.get_node(name='monitor2')        



In [18]:
ssh = server1.get_ssh_command()
print(ssh)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.82


In [19]:
ssh = switch1.get_ssh_command()
command = (
    f"simple_switch -i 0@{switch1_iface1_name} -i 1@{switch1_iface2_name} "
    f"-i 2@{switch_sink_iface1_name} basic.json --log-console &"
)
print(ssh)
print(command)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.164
simple_switch -i 0@enp7s0 -i 1@enp8s0 -i 2@enp9s0 basic.json --log-console &


In [20]:
ssh = switch2.get_ssh_command()
command = (
    f"simple_switch -i 0@{switch2_iface1_name} -i 1@{switch2_iface2_name} "
    f"basic.json --log-console"
)
print(ssh)
print(command)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fea7:8621
simple_switch -i 0@enp8s0 -i 1@enp7s0 basic.json --log-console


In [21]:
ssh = switch3.get_ssh_command()
command = (
    f"simple_switch -i 0@{switch3_iface1_name} -i 1@{switch3_iface2_name} "
    f"-i 2@{switch_sink_iface2_name} basic.json --log-console &"
)
print(ssh)
print(command)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fedb:5f2f
simple_switch -i 0@enp8s0 -i 1@enp9s0 -i 2@enp7s0 basic.json --log-console &


In [22]:
ssh = server2.get_ssh_command()
print(ssh)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:d61:4101:f816:3eff:feeb:a487


In [23]:
ssh = monitor1.get_ssh_command()
print(ssh)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.184


In [24]:
ssh = monitor2.get_ssh_command()
print(ssh)

ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fea0:dcd9


sudo su

p4c INTProject/p4/basic.p4

p4c INTProject/p4Switch1/basic.p4

p4c INTProject/p4Switch2/basic.p4

p4c INTProject/p4Switch3/basic.p4

simple_switch -i 0@enp7s0 -i 1@enp8s0 basic.json --log-console

simple_switch -i 0@enp8s0 -i 1@enp7s0 basic.json --log-console &

simple_switch -i 0@enp8s0 -i 1@enp9s0 basic.json --log-console &

simple_switch -i 0@enp9s0 -i 1@enp8s0 basic.json --log-console &

simple_switch -i 0@enp7s0 -i 1@enp9s0 basic.json --log-console &

simple_switch -i 0@enp9s0 -i 1@enp7s0 basic.json --log-console &

simple_switch -i 0@enp7s0 -i 1@enp9s0 -i 2@enp8s0 basic.json --log-console &

simple_switch -i 0@enp9s0 -i 1@enp7s0 -i 2@enp8s0 basic.json --log-console &

simple_switch -i 0@enp9s0 -i 1@enp8s0 -i 2@enp7s0 basic.json --log-console &

simple_switch -i 0@enp7s0 -i 1@enp8s0 -i 2@enp9s0 basic.json --log-console &

simple_switch -i 0@enp8s0 -i 1@enp9s0 -i 2@enp7s0 basic.json --log-console &


pkill -t pts/1

simple_switch_CLI

pkill simple_switch

mirroring_add 1 2

In [35]:
#Upload p4 programs
switch1 = slice.get_node(name='switch1')        
switch1.upload_directory('p4Switch1', '/home/ubuntu/INTProject')
switch1.upload_directory('p4', '/home/ubuntu/INTProject')

switch2 = slice.get_node(name='switch2')        
switch2.upload_directory('p4Switch2', '/home/ubuntu/INTProject')
switch2.upload_directory('p4', '/home/ubuntu/INTProject')

switch3 = slice.get_node(name='switch3')        
switch3.upload_directory('p4', '/home/ubuntu/INTProject')
switch3.upload_directory('p4Switch3', '/home/ubuntu/INTProject')

'success'

In [36]:
#Upload rules 
#Make sure programs are running!
switch1.upload_file('rules/switch1Rules.sh', 'rules.sh')
stdout, stderr = switch1.execute('chmod +x rules.sh && ./rules.sh')

switch2.upload_file('rules/switch2Rules.sh', 'rules.sh')
stdout, stderr = switch2.execute('chmod +x rules.sh && ./rules.sh')

switch3.upload_file('rules/switch3Rules.sh', 'rules.sh')
stdout, stderr = switch3.execute('chmod +x rules.sh && ./rules.sh')

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:01:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:01	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to exact match table MyIngress.ipv4_exact
match key:           EXACT-c0:a8:01:01
action:              MyIngress.forward
runtime data:        00:00:00:00:00:01	00:00
Entry has been added with handle 0
RuntimeCmd: 
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-c0:a8:04:00/24
action:              MyIngress.forward
runtime data:        00:00:00:00:00:04	00:01
Entry has been added with handle 1
RuntimeCmd: 
Obtaining JSON from switch...
Done
Co

In [29]:
monitor1 = slice.get_node(name='monitor1')        
monitor1.upload_file('monitor.py', 'monitor.py')
monitor2 = slice.get_node(name='monitor2')        
monitor2.upload_file('monitor.py', 'monitor.py')

<SFTPAttributes: [ size=724 uid=1000 gid=1000 mode=0o100664 atime=1733556714 mtime=1733556714 ]>

In [153]:
import time
server1.upload_file('p4/send.py', 'send.py')
#server1.execute(f'sudo python3 send.py {server1_iface} 192.168.1.10 192.168.2.10 HelloWorld')
stdout, stderr = server1.execute(f'sudo python3 send.py enp7s0 192.168.1.1 192.168.4.2 HelloWorld')
#    time.sleep(0.5)

sending on interface enp7s0 to 192.168.4.2
###[ Ethernet ]### 
  dst       = 00:00:00:00:00:02
  src       = 00:00:00:00:00:01
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 51
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xf470
     src       = 192.168.1.1
     dst       = 192.168.4.2
     \options   \
###[ TCP ]### 
        sport     = 52253
        dport     = 1234
        seq       = 0
        ack       = 0
        dataofs   = 5
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = 0x99a
        urgptr    = 0
        options   = []
###[ Raw ]### 
           load      = '192.168.1.1'



sudo sysctl -w net.ipv6.conf.all.disable_ipv6=1
sudo sysctl -w net.ipv6.conf.default.disable_ipv6=1
sudo sysctl -w net.ipv6.conf.lo.disable_ipv6=1

tshark -i enp7s0 -w captures/newCapture.pcap

chmod 644 /home/ubuntu/captures/newCapture.pcap

scp -r -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2620:0:d61:4101:f816:3eff:fec0:4850]:~/captures ~/work/Project/captures

scp -r -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@[2001:468:c00:ffc4:f816:3eff:fe44:88d]:~/telemetry_data.csv ~/work/Project/telemetry

scp -r -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.164:~/captures ~/work/Project/captures

132.249.252.157

iperf3 -c 192.168.4.2 -u -b 100G -t 120 -P 10 -l 1000
iperf3 -c 192.168.4.2 -u -b 10G -t 30 -l 1000
iperf3 -c 192.168.4.2 -u -b 1M -t 30 -l 1000

In [180]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set end host to now plus 1 day
end_date = (datetime.now(timezone.utc) + timedelta(days=8)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name="INTProject")

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")


Retry: 0, Time: 90 sec


ID,ed496368-791e-47bf-84c3-0a897d6004a9
Name,INTProject
Lease Expiration (UTC),2024-12-09 10:03:01 +0000
Lease Start (UTC),2024-12-01 20:26:07 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
7f094f23-882b-4d98-b281-809c591a9fb0,monitor1,4,8,100,default_ubuntu_20,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.187,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.187,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8e3fad1b-70df-4160-8ed7-bf4371038a91,monitor2,4,8,100,default_ubuntu_20,qcow2,atla-w1.fabric-testbed.net,ATLA,ubuntu,2001:400:a100:3050:f816:3eff:fefd:2458,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3050:f816:3eff:fefd:2458,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1afb954b-d143-4583-9dde-f77cdb21c27d,server1,4,8,100,default_ubuntu_20,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.131,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.131,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c27471fb-0c64-4782-a36a-3d56b2fcbcf8,server2,4,8,100,default_ubuntu_20,qcow2,rutg-w4.fabric-testbed.net,RUTG,ubuntu,2620:0:d61:4101:f816:3eff:fec0:4850,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:0:d61:4101:f816:3eff:fec0:4850,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6ebf4c0b-cf9b-4ab2-b0fc-114a7340dbbb,switch1,16,16,100,default_ubuntu_20,qcow2,ucsd-w3.fabric-testbed.net,UCSD,ubuntu,132.249.252.150,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.150,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
3ae94e42-ffe5-4452-856f-e7d203badcc4,switch2,16,16,100,default_ubuntu_20,qcow2,star-w1.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe09:9748,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe09:9748,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
52ef7ca9-7a1c-4ffd-8aaf-07e85485bfab,switch3,16,16,100,default_ubuntu_20,qcow2,atla-w1.fabric-testbed.net,ATLA,ubuntu,2001:400:a100:3050:f816:3eff:fefb:b592,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3050:f816:3eff:fefb:b592,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b8c6f6bb-b9a4-4e24-9ceb-178e23ad719a,net1,L2,L2STS,None,None,None,Active,
61a9065f-433d-4d65-80b0-ae33f5d64b12,net2,L2,L2STS,None,None,None,Active,
4d2a04ad-50b1-4fb8-881f-e60e2a41b8d3,net3,L2,L2STS,None,None,None,Active,
dfc20abb-9d50-45de-adce-9af4e9e1bf70,net4,L2,L2STS,None,None,None,Active,
35774493-8929-414d-a45d-96944d8df864,net5,L2,L2Bridge,UCSD,None,None,Active,
8cf09cf5-953f-4fff-8a52-fa788818e6fa,net6,L2,L2Bridge,ATLA,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
server1-None-p1,p1,server1,net1,100,config,,06:5C:64:37:8B:68,None,enp7s0,192.168.1.1,4,HundredGigE0/0/0/9
server2-None-p1,p1,server2,net4,100,config,,7A:3F:F7:2B:73:F1,None,enp7s0,192.168.4.2,4,HundredGigE0/0/0/11
monitor1-None-p1,p1,monitor1,net5,100,config,,0A:E9:2C:69:4F:9A,None,enp7s0,192.168.5.2,4,HundredGigE0/0/0/9
monitor2-None-p1,p1,monitor2,net6,100,config,,1A:53:41:7E:66:D5,None,enp7s0,192.168.6.2,4,HundredGigE0/0/0/5
switch1-net2_nic-p1,p1,switch1,net2,100,config,,06:AE:3B:35:9F:83,None,enp8s0,192.168.2.1,4,HundredGigE0/0/0/9
switch1-net1_nic-p1,p1,switch1,net1,100,config,,0A:20:B9:BE:1A:AC,None,enp9s0,192.168.1.2,4,HundredGigE0/0/0/9
switch1-net3_nic-p1,p1,switch1,net5,100,config,,06:9F:D6:8E:7E:58,None,enp7s0,192.168.5.1,4,HundredGigE0/0/0/9
switch2-net1_nic-p1,p1,switch2,net2,100,config,,0A:A8:0F:C5:D7:18,None,enp7s0,192.168.2.2,6,HundredGigE0/0/0/5
switch2-net2_nic-p1,p1,switch2,net3,100,config,,12:02:E4:72:64:99,None,enp8s0,192.168.3.1,6,HundredGigE0/0/0/5
switch3-net1_nic-p1,p1,switch3,net3,100,config,,1E:6B:36:89:38:57,None,enp9s0,192.168.3.2,4,HundredGigE0/0/0/5



Time to print interfaces 135 seconds


In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)


In [28]:
slice = fablib.get_slice(name="INTProject")
slice.delete()

Plan
-Append test metadata at each switch
-Remove metadata when next hop is destination

-Set up Monitor
-Send removed metadata to monitor